In [24]:
import pandas as pd
import json

def read_eval(path):
    return pd.read_json(path, orient="index")

In [25]:
def get_number_fixed(df):
    count = 0
    for index, row in df.iterrows():
        for fix in row.fixes:
            if fix["identical"] and row.buggy_line != row.fixed_line:
                count += 1
    return "%d / %d (%.2f%%)" % (count, len(df), count/len(df))

In [26]:
def get_table_entry(lr, epochs, early_stop, beam_width, beam_groups, tbb, tbs, tdb, tds, tb, td):
    return {"Hyper-paramters": {
                "Learning Rate" : {"" : lr},
                "Epochs" : {"" : epochs},
                "Early Stop": {"" : early_stop},
                "Beam Search": {"Beam Width" : beam_width, "Beam Groups" : beam_groups},
            },
            "Training": {
                "Bears" : {"BugLab" : tbb, "SelfAPR": tbs},
                "Defects4J" : {"BugLab" : tdb, "SelfAPR": tds}
            },
            "Testing":{
                "Bears" : { "" : tb },
                "Defects4J" : { "" : td}
            }
       }

In [29]:
tb = get_number_fixed(read_eval("./inesc_models/pre_trained_defects4j_buglab_single_line_15epochs/eval_bears_bw_100_bg_1.json"))
td = get_number_fixed(read_eval("./inesc_models/pre_trained_defects4j_buglab_single_line_15epochs/eval_defects4j_bw_100_bg_1.json"))
entries = [get_table_entry(0.0001, 15, True, 100, 1, False, False, True, False, tb, td)]

tb = get_number_fixed(read_eval("./inesc_models/pre_trained_defects4j_buglab_single_line_15epochs/eval_bears_bw_1000_bg_1.json"))
td = "N/A"
entries += [get_table_entry(0.0001, 15, True, 1000, 1, False, False, True, False, tb, td)]


tb = get_number_fixed(read_eval("./inesc_models/pre_trained_bears_buglab_single_line_15epochs/eval_bears_bw_100_bg_1.json"))
td = get_number_fixed(read_eval("./inesc_models/pre_trained_bears_buglab_single_line_15epochs/eval_defects4j_bw_100_bg_1.json"))
entries += [get_table_entry(0.0001, 15, True, 100, 1, True, False, False, False, tb, td)]

tb = get_number_fixed(read_eval("./inesc_models/pre_trained_bears_buglab_single_line_15epochs/eval_bears_bw_1000_bg_1.json"))
td = "N/A"
entries += [get_table_entry(0.0001, 15, True, 1000, 1, True, False, False, False, tb, td)]

In [30]:
table = pd.json_normalize(data=entries)
table.columns = table.columns.str.split(".").map(tuple)
table

Hyper-paramters                                           Training          \
    Learning Rate Epochs Early Stop Beam Search                Bears           
                                     Beam Width Beam Groups   BugLab SelfAPR   
0          0.0001     15       True         100           1    False   False   
1          0.0001     15       True        1000           1    False   False   
2          0.0001     15       True         100           1     True   False   
3          0.0001     15       True        1000           1     True   False   

                            Testing                    
  Defects4J                   Bears         Defects4J  
     BugLab SelfAPR                                    
0      True   False  0 / 20 (0.00%)  11 / 121 (0.09%)  
1      True   False  0 / 20 (0.00%)               N/A  
2     False   False  1 / 20 (0.05%)   7 / 121 (0.06%)  
3     False   False  1 / 20 (0.05%)               N/A